In [ ]:
pip install evaluate

: 

In [3]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 319.0 kB/s eta 0:00:00 kB/s eta 0:00:01


In [4]:
from transformers import logging
logging.set_verbosity_error()

In [5]:
import os
import evaluate

<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject
2024-04-05 14:31:09.193313: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 14:31:09.193375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 14:31:09.195474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-05 14:31:09

In [6]:
bertscore = evaluate.load("bertscore")

In [32]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 2.8 MB/s eta 0:00:00m eta 0:00:010:01:01


# Read data

In [ ]:

path1 = ""


In [200]:
testDataPath = "NewsRoomTestAbs.json"

In [ ]:
import ast
# Read all lines into a list
with open("ArxivTest.json") as f:
    content = f.readlines()

# Convert each list item to a dict
content = [ ast.literal_eval( line ) for line in content ]

In [185]:
# Open the JSON file
with open(testDataPath, 'r') as file:
    # Load the JSON data
    testData = json.load(file)

# Print the loaded data
#print(data)

In [182]:
import json
def read_data(p):
    articles = []
    with open(p, 'r') as file:
                #articles = json.load(file)
                for line in file:
                    try:
                        articles.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")

    return articles

In [201]:
import json
import time
from datetime import timedelta
def load_jsonl(data_path):
    news = []
    start = time.time()
    print("Start Loading Data ")
    with open(data_path) as f:
        for line in f:
            news.append(json.loads(line))
    data = []
    for row in news:
        #data.append({"document":row['text'] , "summary":row['summary']})
        data.append(row)
    #data = data[0:800]
    #print(data[1:10])
    print('finished in {}'.format(timedelta(seconds=time.time()-start)))
    print("# of Documents ",len(data))

    return data

In [202]:
testData = read_data(testDataPath)

In [203]:
testData[0]

{'docID': 0,
 'text': 'Men in battle-fatigues have raided the Moscow headquarters of billionaire investor George Soros.\n\nYekaterina Geniyeva, president of Soros\'s Open Society Institute, said the attackers took away all of its documents in the climax to a long-running commercial dispute.\n\nShe told a radio station on Friday the men identified themselves as working for the Sektor-1 company, which says it owns the city centre building.\n\n"The most terrifying thing was that they took away all our documents. We do not know where," she said, adding the foundation did not recognise Sektor-1\'s ownership rights.\n\nShe said she did not think there was any direct link between the incident and the arrest of oil billionaire Mikhail Khodorkovsky, which some fear may herald a state crackdown on big business.\n\nBut she said there might be "some connection".\n\nGeorge Soros1930 - Born in Hungary 1952- Graduates from London School of Economics1956- Moves to USA 1987 - Sets up Soros Foundation-S

In [160]:
len(testData)

313

In [12]:
# Upload libraries
!pip install transformers
!pip install nltk
!pip install rouge
!pip install bert-score
!pip install scikit-learn
!pip install scipy
!pip install numpy

In [13]:
# If you use colab, you'll need to " ! " for upload 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer
from rouge import Rouge
from rouge_score import rouge_scorer

# Rouge

# ======= Rouge Score =====

In [106]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/walid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sents = sent_tokenize(item2)
if len(sents) > 2:
  item3 = sents[0]+""+sents[1]
else:
   item3 = item2

In [ ]:
import os
import evaluate
import json
rouge = evaluate.load('rouge')
count = 0
shot = "5-shot2"
dataset = "NewsRoomTestAbs"
outputModelsPath = "Final_Output/FewShot/NewsRoomAbs/"+shot
paths = [   #"ArxivTest_FLAN_T5_Large_prompt#"+str(count),
            #dataset+"_gemma-7b-it_prompt#"+str(count),
            dataset+"_Llama-2-7b_prompt#"+str(count), 
            dataset+"_Llama-2-13b_prompt#"+str(count), 
            dataset+"_Llama-2-70b_prompt#"+str(count),
            dataset+"_Mistral-7B-Instruct-v0.1_prompt#"+str(count),
            dataset+"_Mixtral-8x7B-Instruct-v0.1_prompt#"+str(count)]

res = 0
#"FLAN_T5_Large","gemma-7b-it",
models = ["Llama-2-7b","Llama-2-13b","Llama-2-70b","Mistral-7B-Instruct-v0.1","Mixtral-8x7B-Instruct-v0.1"]
for path in paths:
    outputDataPath1 = os.path.join(outputModelsPath, path + str(".json"))

    with open(outputDataPath1, encoding='utf-8') as data_file:
        outputData1 = json.loads(data_file.read())
    ref1 = []
    pred1 = []
    
    for item in outputData1[0:300]:
        docID = item['docID']
        #nextSum = next(item for item in testData if item["docID"] == docID)
        filtered_dicts = [d for d in testData if d.get("docID") == docID]
        if len(filtered_dicts) == 1:
            summary = filtered_dicts[0]['summary']
        #summary = nextSum['summary']
        summary =summary.replace("\n","")
        ref1.append(summary)
        p = item['generated'].strip().replace("\n","")
        
        #p = p[0:120]
        #print(item3)
        #print("=====")
        pred1.append(p)
    
    result1 = rouge.compute(predictions=pred1, references=ref1)
    print("Path = ",path)
    print("Result # ",res,", Prompt # ",count, ", Model = ",models[res])
    print(result1)
    
    print("===========================================================")
    res = res + 1

In [213]:
pred1[200]

"Christie Proposes FedEx-like System to Track Visa Overstays, Differentiates from Trump's Plan."

In [214]:
ref1[200]

'New Jersey Governor Chris Christie pushed back against "ridiculous" criticism of his proposal to track foreign visitors the way FedEx tracks packages.'

# BERT Score

In [ ]:
text1 = "Stars shine, and the sky is covered with a blue blanket."
text2 = "Stars shine, and the sky is adorned with a bright color."

scorer = BERTScorer(model_type='bert-base-uncased')
P, R, F1 = scorer.score([text1], [text2])
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")


In [ ]:
import os
from pprint import pprint
from torchmetrics.text.bert import BERTScore
bertscore = BERTScore()
from bert_score import score
res = 0
count = 3

scorer = BERTScorer(model_type='bert-base-uncased')
#"SamsumTest_FLAN_T5_Large_prompt#"+str(count),
shot = "5-shot"
dataset = "PubMedTest1"
outputModelsPath = "Final_Output/ZeroShot/PubMed/prompt#"+str(count)
paths = [   #"ArxivTest_FLAN_T5_Large_prompt#"+str(count),
            dataset+"_gemma-7b-it_prompt#"+str(count),
            dataset+"_Llama-2-7b_prompt#"+str(count), 
            dataset+"_Llama-2-13b_prompt#"+str(count), 
            dataset+"_Llama-2-70b_prompt#"+str(count),
            dataset+"_Mistral-7B-Instruct-v0.1_prompt#"+str(count),
            dataset+"_Mixtral-8x7B-Instruct-v0.1_prompt#"+str(count)]
#"FLAN_T5_Large",
models = ["gemma-7b-it","Llama-2-7b","Llama-2-13b",
            "Llama-2-70b","Mistral-7B-Instruct-v0.1","Mixtral-8x7B-Instruct-v0.1"]
for path in paths:
    outputDataPath1 = os.path.join(outputModelsPath, path + str(".json"))

    with open(outputDataPath1, encoding='utf-8') as data_file:
        outputData1 = json.loads(data_file.read())
    ref1 = []
    pred1 = []
    
    for item in outputData1[0:300]:
        docID = item['docID']
        """summary = next(item for item in testData if item["docID"] == docID)['summary']
        summary =summary.replace("\n","")"""
        filtered_dicts = [d for d in testData if d.get("docID") == docID]
        if len(filtered_dicts) == 1:
            summary = filtered_dicts[0]['summary']
        ref1.append(summary)
        p = item['generated'].strip().replace("\n","")

        pred1.append(p)
    
    result1 = scorer.score(pred1, ref1)
    P, R, F1 = score(pred1, ref1, lang='en', verbose=False)

    print("Path = ",path.strip(),".json")
    print("Result # ",res,", Prompt # ",count, ", Model = ",models[res])
    print("Precision = ",P.mean().item()," ,Recall = ",R.mean().item()," ,F1 Score = ",F1.mean().item())
    print("===========================================================")
    res = res + 1